In [8]:
# -*- coding: utf-8 -*-
#

from cached_property import cached_property

#from .backend import DataBackend
#from ..utils import lru_cache, get_str_date_from_int, get_int_date
from backend import DataBackend
from utils import lru_cache, get_str_date_from_int, get_int_date


class TushareDataBackend(DataBackend):

    #@cached_property
    def ts(self):
        try:
            import tushare as ts
            return ts
        except ImportError:
            print("-" * 50)
            print(">>> Missing tushare. Please run `pip install tushare`")
            print("-" * 50)
            raise

class QUANTAXISDataBackend(DataBackend):

    #@cached_property
    def QA(self):
        try:
            import QUANTAXIS as QA
            return QA
        except ImportError:
            print("-" * 50)
            print(">>> Missing QUANTAXIS. Please run `pip install QUANTAXIS`")
            print("-" * 50)
            raise
'''            
| 类别    | 实时                 | 从数据库中            | QADataStruct格式|
| --------| ------------------- | --------------- |  --------------- |
| 股票日线 | QA_fetch_get_stock_day | QA_fetch_stock_day | QA_fetch_stock_day_adv |
| 股票分钟线 | QA_fetch_get_stock_min | QA_fetch_stock_min | QA_fetch_stock_min_adv |
| 股票列表 | QA_fetch_get_stock_list | QA_fetch_stock_list | QA_fetch_stock_list_adv |
| 股票版块 | QA_fetch_get_stock_block | QA_fetch_stock_block | QA_fetch_stock_block_adv |
| 股票分笔 | QA_fetch_get_stock_transaction | X| X|
| 股票实时分笔 | QA_fetch_get_stock_transaction_realtime | X| X|
| 股票权息 | QA_fetch_get_stock_xdxr | QA_fetch_stock_xdxr | X|
| 股票信息 | QA_fetch_get_stock_info | QA_fetch_stock_info| X|
| 股票实时5挡价格 | QA_fetch_get_stock_realtime | X| X|
| 指数日线 | QA_fetch_get_index_day | QA_fetch_index_day | QA_fetch_index_day_adv |
| 指数分钟线 | QA_fetch_get_index_min | QA_fetch_index_min | QA_fetch_index_day_min |
| ETF日线 | QA_fetch_get_etf_day | QA_fetch_etf_day | QA_fetch_etf_day_adv |
| ETF分钟线 | QA_fetch_get_etf_min | QA_fetch_etf_min | QA_fetch_etf_min_adv |
| 期货日线 | QA_fetch_get_future_day | X| X|
| 期货分钟线 | QA_fetch_get_future_min | X| X|
| 期货历史分笔 | QA_fetch_get_future_transaction | X| X|
| 期货实时分笔 | QA_fetch_get_future_transaction_realtime | X| X|
| 期货列表 | QA_fetch_get_future_list | X| X|
'''

    #@cached_property
    def stock_basics(self):
        return self.QA.QA_fetch_stock_day_adv()

    #@cached_property
    def code_name_map(self):
        code_name_map = self.stock_basics[["name"]].to_dict()["name"]
        return code_name_map

    def convert_code(self, order_book_id):
        return order_book_id.split(".")[0]

    #@lru_cache(maxsize=4096)
    def get_price(self, order_book_id, start, end, freq):
        """
        :param order_book_id: e.g. 000002.XSHE
        :param start: 20160101
        :param end: 20160201
        :returns:
        :rtype: numpy.rec.array
        """
        start = get_str_date_from_int(start)
        end = get_str_date_from_int(end)
        code = self.convert_code(order_book_id)
        is_index = False
        if ((order_book_id.startswith("0") and order_book_id.endswith(".XSHG")) or
            (order_book_id.startswith("3") and order_book_id.endswith(".XSHE"))
            ):
            is_index = True
        ktype = freq
        if freq[-1] == "m":
            ktype = freq[:-1]
        elif freq == "1d":
            ktype = "D"
        # else W M

        df = self.ts.get_k_data(code, start=start, end=end, index=is_index, ktype=ktype)

        if freq[-1] == "m":
            df["datetime"] = df.apply(
                lambda row: int(row["date"].split(" ")[0].replace("-", "")) * 1000000 + int(row["date"].split(" ")[1].replace(":", "")) * 100, axis=1)
        elif freq in ("1d", "W", "M"):
            df["datetime"] = df["date"].apply(lambda x: int(x.replace("-", "")) * 1000000)

        del df["code"]
        arr = df.to_records()

        return arr

    #@lru_cache()
    def get_order_book_id_list(self):
        """获取所有的股票代码列表
        """
        info = self.ts.get_stock_basics()
        code_list = info.index.sort_values().tolist()
        order_book_id_list = [
            (code + ".XSHG" if code.startswith("6") else code + ".XSHE")
            for code in code_list
        ]
        return order_book_id_list

    @lru_cache()
    def get_trading_dates(self, start, end):
        """获取所有的交易日

        :param start: 20160101
        :param end: 20160201
        """
        start = get_str_date_from_int(start)
        end = get_str_date_from_int(end)
        df = self.ts.get_k_data("000001", index=True, start=start, end=end)
        trading_dates = [get_int_date(date) for date in df.date.tolist()]
        return trading_dates

    @lru_cache(maxsize=4096)
    def symbol(self, order_book_id):
        """获取order_book_id对应的名字
        :param order_book_id str: 股票代码
        :returns: 名字
        :rtype: str
        """
        code = self.convert_code(order_book_id)
        return "{}[{}]".format(order_book_id, self.code_name_map.get(code))




IndentationError: unexpected indent (<ipython-input-8-0bee23f6c101>, line 48)